<a href="https://colab.research.google.com/github/AimanDzaky/Dzaky-projects/blob/main/AI_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

url = "https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv"
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

# Load dataset into DataFrame
data = pd.read_csv(url, header=0, names=column_names)

# Drop any empty rows (if any)
data.dropna(inplace=True)

In [ ]:

# Encode labels
# Map class labels to integers (e.g., 'Iris-setosa' = 0, 'Iris-versicolor' = 1, 'Iris-virginica' = 2)
class_mapping = {label: idx for idx, label in enumerate(data['class'].unique())}
data['class'] = data['class'].map(class_mapping)

# Shuffle the dataset to ensure random distribution across train/test splits
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Extract features and labels
X = data.drop('class', axis=1).values
y = data['class'].values

# Normalize (Z-score)
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std

# Add bias term by appending a column of ones to the features
X = np.hstack((X, np.ones((X.shape[0], 1))))

In [ ]:

# Function to split dataset into train and test sets based on a given ratio
def split_dataset(X, y, train_ratio):
    split_index = int(len(X) * train_ratio)
    return X[:split_index], y[:split_index], X[split_index:], y[split_index:]

# Dataset splits
splits = {
    "80_20": split_dataset(X, y, 0.8),
    "70_30": split_dataset(X, y, 0.7),
    "60_40": split_dataset(X, y, 0.6)
}

In [ ]:
# Initialize weights (zero or small random values)
def init_weights(n_classes, n_features, strategy='zeros'):
    if strategy == 'zeros':
        return np.zeros((n_classes, n_features))
    elif strategy == 'random':
        # Scaled random init
        return np.random.randn(n_classes, n_features) * 0.01 # small random init

# Train the Perceptron model until convergence or max_epochs
def train_perceptron(X, y, n_classes, learning_rate=0.1, init='zeros', max_epochs=10000):
    n_features = X.shape[1]
    W = init_weights(n_classes, n_features, strategy=init)
    for epoch in range(max_epochs):
        errors = 0
        for i in range(len(X)):
            xi = X[i]
            yi = y[i]
            scores = np.dot(W, xi)
            pred = np.argmax(scores)
            # Update weights if prediction is wrong
            if pred != yi:
                W[yi] += learning_rate * xi
                W[pred] -= learning_rate * xi
                errors += 1
        if errors == 0:
            break         # Model has converged
    return W, epoch + 1   # Return trained weights and number of epochs used

# Predict class labels using learned weight
def predict(X, W):
    return np.argmax(np.dot(X, W.T), axis=1)

# Compute classification accuracy
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

# Build confusion matrix manually (no sklearn used)
def confusion_matrix(y_true, y_pred, n_classes):
    cm = np.zeros((n_classes, n_classes), dtype=int)
    for actual, predicted in zip(y_true, y_pred):
        cm[actual][predicted] += 1
    return cm

# Run experiments with confusion matrix
def run_experiments(splits):
    learning_rates = [0.01, 0.1]
    inits = ['zeros', 'random']
    n_classes = 3

    for split_name, (X_train, y_train, X_test, y_test) in splits.items():
        print(f"\n===== Split: {split_name.replace('_', '/')} =====")

        for lr in learning_rates:
            for init in inits:
                print(f"\n[LR={lr}, Init={init}]")
                W, epochs = train_perceptron(X_train, y_train, n_classes=n_classes, learning_rate=lr, init=init)
                y_pred = predict(X_test, W)
                acc = accuracy(y_test, y_pred)
                cm = confusion_matrix(y_test, y_pred, n_classes=n_classes)

                print(f" → Epochs: {epochs}, Accuracy: {acc:.4f}")
                print("Confusion Matrix:")
                print(cm)


# Run
run_experiments(splits)


===== Split: 80/20 =====

[LR=0.01, Init=zeros]
 → Epochs: 10000, Accuracy: 1.0000
Confusion Matrix:
[[ 7  0  0]
 [ 0 11  0]
 [ 0  0 12]]

[LR=0.01, Init=random]
 → Epochs: 10000, Accuracy: 0.9667
Confusion Matrix:
[[ 7  0  0]
 [ 0 11  0]
 [ 0  1 11]]

[LR=0.1, Init=zeros]
 → Epochs: 10000, Accuracy: 1.0000
Confusion Matrix:
[[ 7  0  0]
 [ 0 11  0]
 [ 0  0 12]]

[LR=0.1, Init=random]
 → Epochs: 10000, Accuracy: 1.0000
Confusion Matrix:
[[ 7  0  0]
 [ 0 11  0]
 [ 0  0 12]]

===== Split: 70/30 =====

[LR=0.01, Init=zeros]
 → Epochs: 10000, Accuracy: 0.9556
Confusion Matrix:
[[10  0  0]
 [ 0 17  0]
 [ 0  2 16]]

[LR=0.01, Init=random]
 → Epochs: 10000, Accuracy: 0.9556
Confusion Matrix:
[[10  0  0]
 [ 0 17  0]
 [ 0  2 16]]

[LR=0.1, Init=zeros]
 → Epochs: 10000, Accuracy: 0.9556
Confusion Matrix:
[[10  0  0]
 [ 0 17  0]
 [ 0  2 16]]

[LR=0.1, Init=random]
 → Epochs: 10000, Accuracy: 0.9778
Confusion Matrix:
[[10  0  0]
 [ 0 17  0]
 [ 0  1 17]]

===== Split: 60/40 =====

[LR=0.01, Init=ze

#Label encode
    label_map = {label: idx for idx, label in enumerate(np.unique(y))}
    y_encoded = np.array([label_map[label] for label in y])

    # Normalize features
    X_mean = X.mean(axis=0)
    X_std = X.std(axis=0)
    X_norm = (X - X_mean) / X_std

    return X_norm, y_encoded

# Split dataset into training and testing
def split_data(X, y, train_ratio=0.8):
    np.random.seed(42)
    indices = np.random.permutation(len(X))
    train_size = int(train_ratio * len(X))
    train_idx, test_idx = indices[:train_size], indices[train_size:]
    return X[train_idx], y[train_idx], X[test_idx], y[test_idx]

# Initialize weights
def init_weights(n_classes, n_features, strategy='zeros'):
    if strategy == 'zeros':
        return np.zeros((n_classes, n_features))
    elif strategy == 'random':
        return np.random.uniform(-0.5, 0.5, (n_classes, n_features))

# Train perceptron until convergence
def train_perceptron(X, y, n_classes, learning_rate=0.1, init='zeros', max_epochs=1000):
    n_features = X.shape[1]
    W = init_weights(n_classes, n_features, strategy=init)
    converged = False
    epoch = 0

    while not converged and epoch < max_epochs:
        errors = 0
        for i in range(len(X)):
            xi = X[i]
            yi = y[i]
            scores = np.dot(W, xi)
            pred = np.argmax(scores)

            if pred != yi:
                W[yi] += learning_rate * xi
                W[pred] -= learning_rate * xi
                errors += 1

        if errors == 0:
            converged = True
        epoch += 1

    return W, epoch

# Predict
def predict(X, W):
    return np.argmax(np.dot(X, W.T), axis=1)

# Accuracy
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

# Run experiment with settings
def run_experiments():
    X, y = load_data()
    splits = [0.8, 0.7, 0.6]
    learning_rates = [0.01, 0.1]
    inits = ['zeros', 'random']

    for split in splits:
        print(f"\n===== Split: {int(split*100)}% Train / {int((1-split)*100)}% Test =====")
        X_train, y_train, X_test, y_test = split_data(X, y, train_ratio=split)

        for lr in learning_rates:
            for init in inits:
                print(f"\n[LR={lr}, Init={init}]")
                W, epochs = train_perceptron(X_train, y_train, n_classes=3, learning_rate=lr, init=init)
                y_pred = predict(X_test, W)
                acc = accuracy(y_test, y_pred)
                print(f" → Epochs: {epochs}, Accuracy: {acc:.4f}")

# Execute all experiments
run_experiments()
